Redes Neuronales Convolucionales
=
Clasificación de bananos según su estado de madurez, con el fin de realizar de manera más óptima su cosecha, selección y distribución.

### Definir la red

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Red(nn.Module):

    def __init__(self):
        super(Red, self).__init__()
        # imagen de 3 canales (RGB), 6 canales de salida, dimensiones de 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # calcular total de parámetros por imagen: y = Wx + b
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6) # 6 clases

    # operaciones
    def forward(self, entrada):
        # Capa convolucional C1: recibe 1 imagen de 3 canales (RGB), produce 6 canales de salida, de dimensión 5x5 con activación ReLU
        # y genera un tensor de tamaño (N, 6, 224, 224), donde N es el batch
        c1 = F.relu(self.conv1(entrada))
        # Capa de subsampling S2: max-pooling 2x2, sin parámetros, salida (N, 6, 14, 14), donde N es el batch
        s2 = F.max_pool2d(c1, (2, 2))
        # Capa convolucional C3: 6 canales de entrada, 16 de salida, dimensión de 5x5 con activación ReLU salidas a Tensor de (N, 16, 10, 10)
        c3 = F.relu(self.conv2(s2))
        # Capa de subsampling S4: max-pooling 2x2, sin parámetros, salida (N, 16, 53, 53), donde N es el batch
        s4 = F.max_pool2d(c3, 2)
        #print("Salidas y dimensiones:",s4.shape)
        # Operación Flatten: sin parámetros, salida (44944), donde N es el batch
        s4 = torch.flatten(s4, 1)
        # Capa totalmente conectada F5: entrada (44944), salida (120) con activación ReLU
        f5 = F.relu(self.fc1(s4))
        # Capa totalmente conectada F6: entrada (120), salida (84) con activación ReLU
        f6 = F.relu(self.fc2(f5))
        # Capa de salida (F7): entrada (84), salida (6) -> capa de clasificación
        salida = self.fc3(f6)

        return salida


red = Red()
print(red)

Red(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)


### Cargar dataset
En esta sección se carga todo el dataset de imágenes que se van a utilizar para entrenar, validar y probar el modelo.

In [30]:
# cargar dataset
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import zipfile
import shutil
import os

# descomprimir dataset
ruta = "/content"
rutas = [ruta+"/Banana_Ripeness/train/", ruta+"/Banana_Ripeness/test/"]
with zipfile.ZipFile(ruta+"/Banana_Ripeness.zip", 'r') as zip_ref:
    for ruta in rutas:
        zip_ref.extractall(ruta)

# transformar a tensor y normalizar dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

batch_size = 32
# entrenamiento (70%)
training_data = datasets.ImageFolder(
    root=rutas[0],
    transform=transform
)
# test (15%)
test_data = datasets.ImageFolder(
    root=rutas[1],
    transform=transform
)

# dividir train en validación (15%)
indices = list(range(len(training_data)))
train_indices, val_indices = train_test_split(indices, test_size=0.15, random_state=42, shuffle=True)

# crear variables para datos de entrenamiento, validación y pruebas / DATALOADERS
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

print("Datos de entrenamiento:",training_data)
print("Datos de prueba:",test_data)
# obtener clases
clases=('freshripe','freshunripe','unripe','overripe','ripe','rotten')
print("Clases:\n",clases)

Datos de entrenamiento: Dataset ImageFolder
    Number of datapoints: 23334
    Root location: /content/Banana_Ripeness/train/
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Datos de prueba: Dataset ImageFolder
    Number of datapoints: 23334
    Root location: /content/Banana_Ripeness/test/
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Clases:
 ('freshripe', 'freshunripe', 'unripe', 'overripe', 'ripe', 'rotten')


### Obtener caractertisticas y etiquetas de las imágenes

In [31]:
#obtener caracteristicas de imagenes y etiquetas
train_features, train_labels = next(iter(train_dataloader))
print(f"Tamaño de lote, canales y dimensiones: {train_features.size()}")
print(f"Número de etiquetas por cada lote: {train_labels.size()}")

Tamaño de lote, canales y dimensiones: torch.Size([32, 3, 224, 224])
Número de etiquetas por cada lote: torch.Size([32])


### Definir función de pérdida y optimizador
Mediante está función se optimiza el proceso de aprendizaje de la red a través de la Gradiente Descendente

In [32]:
import torch.optim as optim

criterio = nn.CrossEntropyLoss()
# momentum sirve para acelerar el proceso de encontrar el valor más cercano a cero
optimizador = optim.SGD(red.parameters(), lr=0.001, momentum=0.9)
print(optimizador)

### Configurar uso de GPU
Buscar dispositivo (GPU) disponible para acelerar entrenamiento

In [33]:
# elección de arquitectura
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Usando:",device)

# Mover el modelo a GPU/CPU
red.to(device)

Usando: cuda


Red(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

### Entrenamiento de la CNN
Entrenamiento de la CNN mediante el uso del datos de entrenamiento en un número específico de épocas, dónde por cada una de estas se busca y actualiza el mejor modelo según la pérdida con los datos de validación.

In [35]:
# Entrenamiento de modelo
mejor_modelo = None
menor_perdida = float("inf")
m_epoca = 0
train_perdida = []
train_precision = []

for epoca in range(10):  # número de épocas
    red.train() # entrenamiento
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        entradas, etiquetas = data
        # mover datos a GPU/CPU
        entradas, etiquetas = entradas.to(device), etiquetas.to(device)
        # gradiente de ceros
        optimizador.zero_grad()

        # forward + backward + optimize
        salidas = red(entradas) # envío
        perdida = criterio(salidas, etiquetas) # calculo diferencia o pérdida
        perdida.backward() # Actualiza pesos
        optimizador.step() # optimizo

        # print statistics
        running_loss += perdida.item() # perdida acumulada
    # pérdida promedio por época
    perdida_promedio = running_loss / len(train_dataloader)
    print(f"Epoca {epoca+1}, perdida promedio: {perdida_promedio:.3f}")

    # guardar mejor modelo
    if perdida_promedio < menor_perdida:
        menor_perdida = perdida_promedio
        m_epoca = epoca+1
        mejor_modelo = red.state_dict()
    # almacenar perdidas en train_perdida
    train_perdida.append(perdida_promedio)

    # validación
    red.eval()
    correcto, total, val_perdida = 0, 0, 0.0
    with torch.no_grad():
      for data in val_dataloader:
        entradas, etiquetas = data
        entradas, etiquetas = entradas.to(device), etiquetas.to(device)
        salidas = red(entradas)
        perdida_val = criterio(salidas, etiquetas)
        val_perdida += perdida_val.item()
        _, pred = torch.max(salidas, 1)
        total += etiquetas.size(0)
        correcto += (pred == etiquetas).sum().item()
        #print("Precision:", correcto/total*100)
    # perdida y ganancia por época
    val_perdida /= len(val_dataloader)
    train_acc = 100 * correcto / total
    train_precision.append(train_acc)
    print(f"Precision {train_acc:0.2f}% y perdida {val_perdida:0.2f} en entrenamiento")

print('='*60)
print(f"Modelo con menor perdida: {menor_perdida:0.5f}, Epoca {m_epoca}")
print('Entrenamiento finalizado')

Epoca 1, perdida promedio: 0.430
Epoca 2, perdida promedio: 0.431
Epoca 3, perdida promedio: 0.427
Epoca 4, perdida promedio: 0.427
Epoca 5, perdida promedio: 0.426
Epoca 6, perdida promedio: 0.426
Epoca 7, perdida promedio: 0.424
Epoca 8, perdida promedio: 0.426
Epoca 9, perdida promedio: 0.425
Epoca 10, perdida promedio: 0.426
Modelo con menor perdida: 0.424, Epoca 7
Entrenamiento finalizado


### Guardar mejor modelo

In [36]:
ruta = './mejor_modelo_banana.pth'
torch.save(mejor_modelo, ruta)

### Cargar modelo entrenado

In [38]:
red = Red()
red.load_state_dict(torch.load(ruta, weights_only=True))
print("Modelo cargado")

Modelo cargado


### Probar modelo
En esta sección se prueba el modelo con el fin de conocer la generalización de este ante los datos de test o prueba, y por ende calcular la precisión de este por cada clase.



In [ ]:
# Prueba de modelo por cada clase
correct_pred = {classname: 0 for classname in clases}
total_pred = {classname: 0 for classname in clases}

# no se necesita la gradiente ya que se entrena una sola vez
with torch.no_grad():
    for data in test_dataloader:
        imagenes, etiquetas = data
        salidas = red(imagenes)
        _, predicciones = torch.max(salidas, 1)
        # collección de predicciones correctas por cada clase
        for etiqueta, prediccion in zip(etiquetas, predicciones):
            if etiqueta == prediccion:
                correct_pred[clases[etiqueta]] += 1
            total_pred[clases[etiqueta]] += 1


# print accuracy for each class
for n_clase, conteo_correctas in correct_pred.items():
    accuracy = 100 * float(conteo_correctas) / total_pred[n_clase]
    print(f'Accuracy por clase: {n_clase:5s} es {accuracy:.2f} %')

### Precisión total
En esta sección se calcula la precisión total del modelo para clasificar todas las clases de madurez de banano.

In [ ]:
# calcular precisión en test
correcto = 0
total = 0
# no se necesita la gradiente
with torch.no_grad():
    for data in test_dataloader:
        entradas, etiquetas = data
        # calculate outputs by running images through the network
        salidas = red(entradas)
        # the class with the highest energy is what we choose as prediction
        _, prediccion = torch.max(salidas, 1)
        total += etiquetas.size(0)
        correcto += (prediccion == etiquetas).sum().item()
print(f'Accuracy de la red con {len(test_data)} imagenes de test: {100 * correcto / total:.2f} %')

### Visualización de resultados
En esta sección se muestra la curvas tanto de pérdida como la curva de precisón en entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

# Curva de pérdida
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_perdida, label='Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida')
plt.legend()

# Curva de precisión
plt.subplot(1,2,2)
plt.plot(train_precision, label='Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión (%)')
plt.title('Curva de precisión')
plt.legend()

plt.show()

### Mostrar imágenes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Función para mostrar una imagen con su etiqueta real y predicha
def mostrar_imagen(img, etiqueta_real, etiqueta_pred):
    img = img.squeeze()  # quitar canal si es [1,28,28]
    plt.imshow(img, cmap="gray")
    plt.title(f"Real: {clases[etiqueta_real]}\nPredicho: {clases[etiqueta_pred]}")
    plt.axis("off")

# Mostrar algunas imágenes del test con predicciones
dataiter = iter(test_dataloader)
imagenes, etiquetas = next(dataiter)

# Pasar por la red
salidas = red(imagenes)
_, predicciones = torch.max(salidas, 1)

# Mostrar primeras 6 imágenes
plt.figure(figsize=(12,6))
for i in range(6):
    plt.subplot(2,3,i+1)
    mostrar_imagen(imagenes[i], etiquetas[i].item(), predicciones[i].item())
plt.show()